In [1]:
import enum
import pandas as pd

In [2]:
input_df = pd.read_csv("C:/Dev/TTScout/src/output.csv")

In [3]:
class OutputHeaders(str, enum.Enum):
    Match = "Match #"
    Team = "Team #"
    Position = "Position"
    Scouter = "Scouter"
    Question = "Question"
    Bubble = "Bubble"
    Value = "Value"

    @classmethod
    def get_values(cls):
        return [e.value for e in cls]

In [4]:
transformed_data = []
input_df.reset_index()

new_headers = {}
tracked_team = None
tracked_match = None
tracked_position = None
tracked_scouter = None
n_rows = len(input_df.index)

for index, row in input_df.iterrows():
    row_team, row_match = row[OutputHeaders.Team.value], row[OutputHeaders.Match.value]
    row_pos, row_scouter = row[OutputHeaders.Position.value], row[OutputHeaders.Scouter.value]

    if tracked_match is None:
        tracked_match = row_match
    if tracked_team is None:
        tracked_team = row_team
    if tracked_position is None:
        tracked_position = row_pos
    if tracked_scouter is None:
        tracked_scouter = row_scouter

    if row_team != tracked_team or row_match != tracked_match:
        l_data = [tracked_match, tracked_team, tracked_position, tracked_scouter]
        l_data.extend(new_headers.values())
        transformed_data.append(l_data)

        new_headers = {}
        tracked_team = row_team
        tracked_match = row_match
        tracked_position = row_pos
        tracked_scouter = row_scouter
    elif index >= (n_rows - 1):
        l_data = [row_match, row_team, row_pos, row_scouter]
        new_headers[row[OutputHeaders.Question.value] + row[OutputHeaders.Bubble.value]] = row[OutputHeaders.Value.value]

        l_data.extend(new_headers.values())
        transformed_data.append(l_data)
        break

    new_headers[row[OutputHeaders.Question.value] + row[OutputHeaders.Bubble.value]] = row[OutputHeaders.Value.value]

# print(transformed_data)
l_headers = [OutputHeaders.Match.value, OutputHeaders.Team.value, OutputHeaders.Position.value, OutputHeaders.Scouter.value]
l_headers.extend(new_headers.keys())
transformed_df = pd.DataFrame(transformed_data, columns=l_headers)

In [5]:
transformed_df.to_csv("output.csv")